In [1]:
from experta import Fact, Rule, KnowledgeEngine, Field, DefFacts, AS, factlist, MATCH, TEST, AND, OR, P, L, W, EXISTS, NOT, FORALL
import schema

In [2]:
class Animal(Fact):
    name = Field(str, mandatory=True)
    approx_weight = Field(int, mandatory=True)
    approx_height = Field(int, mandatory=True)
    tail = Field(int, mandatory=True)
    character = Field(str, mandatory=True)
    water = Field(bool, mandatory=True)
    related_animals = Field(list, mandatory=True)

In [3]:
class Person(Fact):
    name = Field(str, mandatory=False)
    animal_kind = Field(str, mandatory=True)
    age = Field(int, mandatory=True)
    friends = Field(list, mandatory=False)

In [4]:
class Lab(KnowledgeEngine):
    @DefFacts()
    def init_pets(self):
        yield Animal(name='dog', approx_weight=20, approx_height=30, tail=1, character='strong', water=False, land=True, related_animals=['cat', 'wolf'])
        yield Animal(name='cat', approx_weight=4, approx_height=10, tail=1, character='sweet', water=False, land=True, related_animals=['wolf', 'dog'])      
        yield Animal(name='wolf', approx_weight=4, approx_height=40, tail=1, character='bad', water=False, land=True, related_animals=['wolf', 'cat'])      

    
    @DefFacts()
    def init_persons(self):
        yield Person(name="Rick", animal_kind='dog', age=3, friends=['Tim', 'Kitty'])
        yield Person(name="Tim", animal_kind='dog', age=4, friends=['Rick'])
        yield Person(name="Kitty", animal_kind='dog', age=4, friends=['Rick'])
        yield Person(animal_kind='mollusc', age=1)
    
    @Rule(
        Animal(approx_height=MATCH.approx_height1, name=MATCH.n1),
        Animal(approx_height=MATCH.approx_height2, name=MATCH.n2),
        TEST(lambda approx_height1, approx_height2: approx_height1 > approx_height2)
    )
    def testRule(self, n1, n2):
        print(n1, 'higest than', n2)
        
    @Rule(
        Animal(approx_weight=MATCH.approx1, name=MATCH.n1, tail=1),
        Animal(approx_weight=MATCH.approx2, name=MATCH.n2, tail=1),
        TEST(lambda approx1, approx2: approx1 >= approx2)
    )
    def testRule1(self, n1, n2):
        print(n1, 'more heavily than', n2)
        
    
    @Rule(OR(Animal(tail=1, name=MATCH.name),
            Animal(water=True, name=MATCH.name))
    )
    def orAndNotRule(self, name):
        print('orAndNotRule', name)
        
    @Rule(OR(
        Person(name=MATCH.name, animal_kind='cat'),
        Person(name=MATCH.name, friends=P(lambda x: "Kitty" in x))
    ))
    def orAndNotRule1(self, name):
        print("orAndNotRule1", name)
     

    @Rule(OR(
        AND(Animal(name=MATCH.name, character='bad'),
                   Animal(name=MATCH.name, water=~L(True))
           ),
        Animal(name=MATCH.name, related_animals=P(lambda x: 'cat' in x))
    ))
    def orAndNotRule2(self, name):
        print("orAndNotRule2", name)
        
    
    @Rule(
        EXISTS(Animal(related_animals=P(lambda x: 'dog' in x)))
    )
    def existsRule(self):
        print("existsRule: Found tailed dog’s relatives")
    
    
    @Rule(
        NOT(NOT((Animal(related_animals=P(lambda x: 'dog' in x)))))
    )
    def existsRule1(self):
        print("existsRule1: Found tailed dog’s relatives")
        
        
    @Rule(
        FORALL(
            Animal(tail=1),
            Animal(land=True)
        )
    )
    def forAllRule(self):
        print('ForAllRule: All land animals are tailed')
        
     
    @Rule(
        NOT(
            AND(
                Animal(tail=1),
                NOT(Animal(land=True))
            )
        )
    )   
    def ForAllRule1(self):
        print('ForAllRule1: All land animals are tailed')

In [5]:
eng = Lab()
eng.reset()

In [6]:
eng.run()

wolf higest than cat
wolf more heavily than cat
cat more heavily than wolf
wolf higest than dog
dog more heavily than wolf
ForAllRule1: All land animals are tailed
wolf more heavily than wolf
orAndNotRule2 wolf
orAndNotRule2 wolf
orAndNotRule wolf
dog higest than cat
dog more heavily than cat
existsRule1: Found tailed dog’s relatives
ForAllRule1: All land animals are tailed
cat more heavily than cat
orAndNotRule cat
ForAllRule1: All land animals are tailed
dog more heavily than dog
orAndNotRule2 dog
orAndNotRule dog
orAndNotRule1 Rick
existsRule: Found tailed dog’s relatives
ForAllRule: All land animals are tailed


In [7]:
eng.facts.clear()
eng.facts

FactList()